In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import os
import sqlalchemy
from dotenv import load_dotenv
load_dotenv()

True

## **Data Extraction**

In [31]:
# Connect to staging area (CRM)
staging_CRM_URL = os.getenv("STAGING_CRM_URL")
engine_crm = sqlalchemy.create_engine(staging_CRM_URL, client_encoding='utf8', pool_pre_ping=True)
connection_crm = engine_crm.connect()

# Check database tables (CRM)
CRM_tables = sqlalchemy.inspect(engine_crm)
print(CRM_tables.get_table_names())

['CRM_cust_info', 'CRM_prd_info', 'CRM_sales_info']


In [32]:
# Convert to DataFrames (CRM)
CRM_cust_info = pd.read_sql('SELECT * FROM "CRM_cust_info";', con=connection_crm)
CRM_prd_info = pd.read_sql('SELECT * FROM "CRM_prd_info";', con=connection_crm)
CRM_sales_info = pd.read_sql('SELECT * FROM "CRM_sales_info";', con=connection_crm)

In [33]:
# Close connections and dispose engines
connection_crm.close()
engine_crm.dispose()

In [34]:
# Connect to staging area (ERP)
staging_ERP_URL = os.getenv("STAGING_ERP_URL")
engine_erp = sqlalchemy.create_engine(staging_ERP_URL,  client_encoding='utf8')
connection_erp = engine_erp.connect()

# Check database tables (ERP)
ERP_tables = sqlalchemy.inspect(engine_erp)
print(ERP_tables.get_table_names())

['ERP_cust', 'ERP_loc', 'ERP_px']


In [35]:
# Convert to DataFrames (ERP)
ERP_cust = pd.read_sql('SELECT * FROM "ERP_cust";', con=connection_erp)
ERP_loc = pd.read_sql('SELECT * FROM "ERP_loc";', con=connection_erp)
ERP_px = pd.read_sql('SELECT * FROM "ERP_px";', con=connection_erp)

In [36]:
# Close connections and dispose engines
connection_erp.close()
engine_erp.dispose()

## **Data Transformation**

### **For CRM**

In [37]:
CRM_cust_info.head()

,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,11000.0,AW00011000,Jon,Yang,M,M,2025-10-06
1,11001.0,AW00011001,Eugene,Huang,S,M,2025-10-06
2,11002.0,AW00011002,Ruben,Torres,M,M,2025-10-06
3,11003.0,AW00011003,Christy,Zhu,S,F,2025-10-06
4,11004.0,AW00011004,Elizabeth,Johnson,S,F,2025-10-06


In [38]:
CRM_cust_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18494 entries, 0 to 18493
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cst_id              18490 non-null  float64
 1   cst_key             18494 non-null  object 
 2   cst_firstname       18486 non-null  object 
 3   cst_lastname        18487 non-null  object 
 4   cst_marital_status  18487 non-null  object 
 5   cst_gndr            13916 non-null  object 
 6   cst_create_date     18490 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1011.5+ KB


In [39]:
# Replace null values
CRM_cust_info.fillna(0, inplace=True)
CRM_prd_info.fillna(0, inplace=True)
CRM_sales_info.fillna(0, inplace=True)

# Fix data types
CRM_cust_info['cst_id'] = CRM_cust_info['cst_id'].astype(int)
CRM_cust_info['cst_create_date'] = pd.to_datetime(CRM_cust_info['cst_create_date'], format='mixed', errors='coerce')
CRM_cust_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18494 entries, 0 to 18493
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   cst_id              18494 non-null  int64         
 1   cst_key             18494 non-null  object        
 2   cst_firstname       18494 non-null  object        
 3   cst_lastname        18494 non-null  object        
 4   cst_marital_status  18494 non-null  object        
 5   cst_gndr            18494 non-null  object        
 6   cst_create_date     18494 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1011.5+ KB


In [40]:
CRM_prd_info.head()

,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,210,CO-RF-FR-R92B-58,HL Road Frame - Black- 58,0.0,R,2003-07-01,0
1,211,CO-RF-FR-R92R-58,HL Road Frame - Red- 58,0.0,R,2003-07-01,0
2,212,AC-HE-HL-U509-R,Sport-100 Helmet- Red,12.0,S,2011-07-01,2007-12-28
3,213,AC-HE-HL-U509-R,Sport-100 Helmet- Red,14.0,S,2012-07-01,2008-12-27
4,214,AC-HE-HL-U509-R,Sport-100 Helmet- Red,13.0,S,2013-07-01,0


In [41]:
CRM_prd_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   prd_id        397 non-null    int64  
 1   prd_key       397 non-null    object 
 2   prd_nm        397 non-null    object 
 3   prd_cost      397 non-null    float64
 4   prd_line      397 non-null    object 
 5   prd_start_dt  397 non-null    object 
 6   prd_end_dt    397 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 21.8+ KB


In [42]:
# Fix data types
CRM_prd_info['prd_start_dt'] = pd.to_datetime(CRM_prd_info['prd_start_dt'], format='mixed', errors='coerce')
CRM_prd_info['prd_end_dt'] = pd.to_datetime(CRM_prd_info['prd_end_dt'], format='mixed', errors='coerce')
CRM_prd_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   prd_id        397 non-null    int64         
 1   prd_key       397 non-null    object        
 2   prd_nm        397 non-null    object        
 3   prd_cost      397 non-null    float64       
 4   prd_line      397 non-null    object        
 5   prd_start_dt  397 non-null    datetime64[ns]
 6   prd_end_dt    397 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 21.8+ KB


In [43]:
CRM_sales_info.head()

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price
0,SO43697,BK-R93R-62,21768,20101229,20110105,20110110,3578.0,1,3578.0
1,SO43698,BK-M82S-44,28389,20101229,20110105,20110110,3400.0,1,3400.0
2,SO43699,BK-M82S-44,25863,20101229,20110105,20110110,3400.0,1,3400.0
3,SO43700,BK-R50B-62,14501,20101229,20110105,20110110,699.0,1,699.0
4,SO43701,BK-M82S-44,11003,20101229,20110105,20110110,3400.0,1,3400.0


In [44]:
CRM_sales_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sls_ord_num   60398 non-null  object 
 1   sls_prd_key   60398 non-null  object 
 2   sls_cust_id   60398 non-null  int64  
 3   sls_order_dt  60398 non-null  int64  
 4   sls_ship_dt   60398 non-null  int64  
 5   sls_due_dt    60398 non-null  int64  
 6   sls_sales     60398 non-null  float64
 7   sls_quantity  60398 non-null  int64  
 8   sls_price     60398 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 4.1+ MB


In [45]:
# Fix data types
CRM_sales_info['sls_order_dt'] = pd.to_datetime(CRM_sales_info['sls_order_dt'].astype(str), format='%Y%m%d', errors='coerce')
CRM_sales_info['sls_ship_dt'] = pd.to_datetime(CRM_sales_info['sls_ship_dt'].astype(str), format='%Y%m%d', errors='coerce')
CRM_sales_info['sls_due_dt'] = pd.to_datetime(CRM_sales_info['sls_due_dt'].astype(str), format='%Y%m%d', errors='coerce')
CRM_sales_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   sls_ord_num   60398 non-null  object        
 1   sls_prd_key   60398 non-null  object        
 2   sls_cust_id   60398 non-null  int64         
 3   sls_order_dt  60379 non-null  datetime64[ns]
 4   sls_ship_dt   60398 non-null  datetime64[ns]
 5   sls_due_dt    60398 non-null  datetime64[ns]
 6   sls_sales     60398 non-null  float64       
 7   sls_quantity  60398 non-null  int64         
 8   sls_price     60398 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(2), object(2)
memory usage: 4.1+ MB


In [46]:
# Merge Sales and Customer DataFrames
CRM_sales_cust = CRM_sales_info.merge(CRM_cust_info, how='left', left_on='sls_cust_id', right_on='cst_id')
CRM_sales_cust

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
0,SO43697,BK-R93R-62,21768,2010-12-29,2011-01-05,2011-01-10,3578.0,1,3578.0,21768,AW00021768,Cole,Watson,S,M,2026-01-05
1,SO43698,BK-M82S-44,28389,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,0,2026-01-25
2,SO43699,BK-M82S-44,25863,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,F,2026-01-14
3,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12
4,SO43701,BK-M82S-44,11003,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,11003,AW00011003,Christy,Zhu,S,F,2025-10-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60402,SO75122,FE-6654,15868,2014-01-28,2014-02-04,2014-02-09,22.0,1,22.0,15868,AW00015868,Caleb,Lal,S,M,2026-01-04
60403,SO75122,CA-1098,15868,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,15868,AW00015868,Caleb,Lal,S,M,2026-01-04
60404,SO75123,FE-6654,18759,2014-01-28,2014-02-04,2014-02-09,22.0,1,22.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04
60405,SO75123,ST-1401,18759,2014-01-28,2014-02-04,2014-02-09,159.0,1,159.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04


In [47]:
# Fix product key format
CRM_prd_info['prd_key'] = CRM_prd_info['prd_key'].astype(str)
CRM_prd_info['prd_key'] = CRM_prd_info['prd_key'].str.split('-').str[2:]
CRM_prd_info['prd_key'] = CRM_prd_info['prd_key'].apply(lambda x: '-'.join(x))
CRM_prd_info.head()

,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,210,FR-R92B-58,HL Road Frame - Black- 58,0.0,R,2003-07-01,1970-01-01
1,211,FR-R92R-58,HL Road Frame - Red- 58,0.0,R,2003-07-01,1970-01-01
2,212,HL-U509-R,Sport-100 Helmet- Red,12.0,S,2011-07-01,2007-12-28
3,213,HL-U509-R,Sport-100 Helmet- Red,14.0,S,2012-07-01,2008-12-27
4,214,HL-U509-R,Sport-100 Helmet- Red,13.0,S,2013-07-01,1970-01-01


In [48]:
# Merge Sales_Customer and Product DataFrames
CRM_sales_cust_prod = CRM_sales_cust.merge(CRM_prd_info, how='left', left_on='sls_prd_key', right_on='prd_key')
CRM_sales_cust_prod

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt
0,SO43697,BK-R93R-62,21768,2010-12-29,2011-01-05,2011-01-10,3578.0,1,3578.0,21768,AW00021768,Cole,Watson,S,M,2026-01-05,310,BK-R93R-62,Road-150 Red- 62,2171.0,R,2011-07-01,2007-12-28
1,SO43698,BK-M82S-44,28389,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,0,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28
2,SO43699,BK-M82S-44,25863,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,F,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28
3,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,336,BK-R50B-62,Road-650 Black- 62,413.0,R,2011-07-01,2007-12-28
4,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,337,BK-R50B-62,Road-650 Black- 62,487.0,R,2012-07-01,2008-12-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89828,SO75123,FE-6654,18759,2014-01-28,2014-02-04,2014-02-09,22.0,1,22.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,485,FE-6654,Fender Set - Mountain,8.0,M,2013-07-01,1970-01-01
89829,SO75123,ST-1401,18759,2014-01-28,2014-02-04,2014-02-09,159.0,1,159.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,486,ST-1401,All-Purpose Bike Stand,59.0,M,2013-07-01,1970-01-01
89830,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,223,CA-1098,AWC Logo Cap,6.0,S,2011-07-01,2007-12-28
89831,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,224,CA-1098,AWC Logo Cap,5.0,S,2012-07-01,2008-12-27


In [49]:
CRM_sales_cust_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89833 entries, 0 to 89832
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   sls_ord_num         89833 non-null  object        
 1   sls_prd_key         89833 non-null  object        
 2   sls_cust_id         89833 non-null  int64         
 3   sls_order_dt        89809 non-null  datetime64[ns]
 4   sls_ship_dt         89833 non-null  datetime64[ns]
 5   sls_due_dt          89833 non-null  datetime64[ns]
 6   sls_sales           89833 non-null  float64       
 7   sls_quantity        89833 non-null  int64         
 8   sls_price           89833 non-null  float64       
 9   cst_id              89833 non-null  int64         
 10  cst_key             89833 non-null  object        
 11  cst_firstname       89833 non-null  object        
 12  cst_lastname        89833 non-null  object        
 13  cst_marital_status  89833 non-null  object    

### **For ERP**

In [50]:
ERP_cust.head()

,CID,BDATE,GEN
0,NASAW00011000,1971-10-06,Male
1,NASAW00011001,1976-05-10,Male
2,NASAW00011002,1971-02-09,Male
3,NASAW00011003,1973-08-14,Female
4,NASAW00011004,1979-08-05,Female


In [51]:
ERP_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CID     18484 non-null  object
 1   BDATE   18484 non-null  object
 2   GEN     17012 non-null  object
dtypes: object(3)
memory usage: 433.3+ KB


In [52]:
# Fix data type
ERP_cust['BDATE'] = pd.to_datetime(ERP_cust['BDATE'], format='%Y-%m-%d', errors='coerce')

In [53]:
ERP_loc.head()

,CID,CNTRY
0,AW-00011000,Australia
1,AW-00011001,Australia
2,AW-00011002,Australia
3,AW-00011003,Australia
4,AW-00011004,Australia


In [54]:
ERP_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CID     18484 non-null  object
 1   CNTRY   18152 non-null  object
dtypes: object(2)
memory usage: 288.9+ KB


In [55]:
ERP_px.head()

,ID,CAT,SUBCAT,MAINTENANCE
0,AC_BR,Accessories,Bike Racks,Yes
1,AC_BS,Accessories,Bike Stands,No
2,AC_BC,Accessories,Bottles and Cages,No
3,AC_CL,Accessories,Cleaners,Yes
4,AC_FE,Accessories,Fenders,No


In [56]:
ERP_px.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           37 non-null     object
 1   CAT          37 non-null     object
 2   SUBCAT       37 non-null     object
 3   MAINTENANCE  37 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


In [57]:
# Fix customer ID format
ERP_loc['CID'] = ERP_loc['CID'].astype(str)
ERP_loc['CID'] = ERP_loc['CID'].str.split('-')
ERP_loc['CID'] = ERP_loc['CID'].apply(lambda x: ''.join(x))
ERP_loc.head()

,CID,CNTRY
0,AW00011000,Australia
1,AW00011001,Australia
2,AW00011002,Australia
3,AW00011003,Australia
4,AW00011004,Australia


In [58]:
# Merge Customer and Location DataFrames
ERP_cust_loc = ERP_cust.merge(ERP_loc, how='left', left_on='CID', right_on='CID')
ERP_cust_loc

,CID,BDATE,GEN,CNTRY
0,NASAW00011000,1971-10-06,Male,NaN
1,NASAW00011001,1976-05-10,Male,NaN
2,NASAW00011002,1971-02-09,Male,NaN
3,NASAW00011003,1973-08-14,Female,NaN
4,NASAW00011004,1979-08-05,Female,NaN
...,...,...,...,...
18479,AW00029479,1969-06-30,None,France
18480,AW00029480,1977-05-06,None,United Kingdom
18481,AW00029481,1965-07-04,None,Germany
18482,AW00029482,1964-09-01,None,France


In [59]:
ERP_cust_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   CID     18484 non-null  object        
 1   BDATE   18479 non-null  datetime64[ns]
 2   GEN     17012 non-null  object        
 3   CNTRY   7442 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 577.8+ KB


### **Merge CRM and ERP**

In [60]:
# Create keyword dictionary
keyword_subcat_map = {
    'Mountain': 'Mountain Bikes',
    'Road': 'Road Bikes',
    'Touring': 'Touring Bikes',
    'Tire': 'Tires and Tubes',
    'Tube': 'Tires and Tubes',
    'Bottle': 'Bottles and Cages',
    'Cage': 'Bottles and Cages',
    'Helmet': 'Helmets',
    'Gloves': 'Gloves',
    'Jersey': 'Jerseys',
    'Socks': 'Socks',
    'Cap': 'Caps',
    'Vest': 'Vests',
    'Shorts': 'Shorts',
    'Bib-Shorts': 'Bib-Shorts',
    'Hydration Pack': 'Hydration Packs',
    'Bike Wash': 'Cleaners',
    'Patch': 'Cleaners',
    'Fender': 'Fenders',
    'Stand': 'Bike Stands',
    'Rack': 'Bike Racks',
}

# Function to assign subcategory based on the keyword found
def assign_subcat(prd_name, keyword_map):
    for keyword, subcat in keyword_map.items():
        if keyword.lower() in prd_name.lower():
            return subcat
    return 'Unknown'

# Map to subcategory
CRM_sales_cust_prod['subcat'] = CRM_sales_cust_prod['prd_nm'].apply(lambda x: assign_subcat(x, keyword_subcat_map))
CRM_sales_cust_prod

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt,subcat
0,SO43697,BK-R93R-62,21768,2010-12-29,2011-01-05,2011-01-10,3578.0,1,3578.0,21768,AW00021768,Cole,Watson,S,M,2026-01-05,310,BK-R93R-62,Road-150 Red- 62,2171.0,R,2011-07-01,2007-12-28,Road Bikes
1,SO43698,BK-M82S-44,28389,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,0,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes
2,SO43699,BK-M82S-44,25863,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,F,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes
3,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,336,BK-R50B-62,Road-650 Black- 62,413.0,R,2011-07-01,2007-12-28,Road Bikes
4,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,337,BK-R50B-62,Road-650 Black- 62,487.0,R,2012-07-01,2008-12-27,Road Bikes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89828,SO75123,FE-6654,18759,2014-01-28,2014-02-04,2014-02-09,22.0,1,22.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,485,FE-6654,Fender Set - Mountain,8.0,M,2013-07-01,1970-01-01,Mountain Bikes
89829,SO75123,ST-1401,18759,2014-01-28,2014-02-04,2014-02-09,159.0,1,159.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,486,ST-1401,All-Purpose Bike Stand,59.0,M,2013-07-01,1970-01-01,Bike Stands
89830,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,223,CA-1098,AWC Logo Cap,6.0,S,2011-07-01,2007-12-28,Caps
89831,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,224,CA-1098,AWC Logo Cap,5.0,S,2012-07-01,2008-12-27,Caps


In [61]:
# Merge CRM final and ERP Product DataFrames
CRM_ERP_prod = CRM_sales_cust_prod.merge(ERP_px, how='left', left_on='subcat', right_on='SUBCAT')
CRM_ERP_prod

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt,subcat,ID,CAT,SUBCAT,MAINTENANCE
0,SO43697,BK-R93R-62,21768,2010-12-29,2011-01-05,2011-01-10,3578.0,1,3578.0,21768,AW00021768,Cole,Watson,S,M,2026-01-05,310,BK-R93R-62,Road-150 Red- 62,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Road Bikes,Yes
1,SO43698,BK-M82S-44,28389,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,0,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Mountain Bikes,Yes
2,SO43699,BK-M82S-44,25863,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,F,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Mountain Bikes,Yes
3,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,336,BK-R50B-62,Road-650 Black- 62,413.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Road Bikes,Yes
4,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,337,BK-R50B-62,Road-650 Black- 62,487.0,R,2012-07-01,2008-12-27,Road Bikes,BI_RB,Bikes,Road Bikes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89828,SO75123,FE-6654,18759,2014-01-28,2014-02-04,2014-02-09,22.0,1,22.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,485,FE-6654,Fender Set - Mountain,8.0,M,2013-07-01,1970-01-01,Mountain Bikes,BI_MB,Bikes,Mountain Bikes,Yes
89829,SO75123,ST-1401,18759,2014-01-28,2014-02-04,2014-02-09,159.0,1,159.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,486,ST-1401,All-Purpose Bike Stand,59.0,M,2013-07-01,1970-01-01,Bike Stands,AC_BS,Accessories,Bike Stands,No
89830,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,223,CA-1098,AWC Logo Cap,6.0,S,2011-07-01,2007-12-28,Caps,CL_CA,Clothing,Caps,No
89831,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,224,CA-1098,AWC Logo Cap,5.0,S,2012-07-01,2008-12-27,Caps,CL_CA,Clothing,Caps,No


In [62]:
# Merge CRM and ERP DataFrames
CRM_ERP = CRM_ERP_prod.merge(ERP_cust_loc, how='left', left_on='cst_key', right_on='CID')
CRM_ERP

,sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt,subcat,ID,CAT,SUBCAT,MAINTENANCE,CID,BDATE,GEN,CNTRY
0,SO43697,BK-R93R-62,21768,2010-12-29,2011-01-05,2011-01-10,3578.0,1,3578.0,21768,AW00021768,Cole,Watson,S,M,2026-01-05,310,BK-R93R-62,Road-150 Red- 62,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Road Bikes,Yes,NaN,NaT,NaN,NaN
1,SO43698,BK-M82S-44,28389,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,0,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Mountain Bikes,Yes,AW00028389,1970-06-17,Female,France
2,SO43699,BK-M82S-44,25863,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,F,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Mountain Bikes,Yes,AW00025863,1952-06-01,Female,United States
3,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,336,BK-R50B-62,Road-650 Black- 62,413.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Road Bikes,Yes,NaN,NaT,NaN,NaN
4,SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699.0,1,699.0,14501,AW00014501,Ruben,Prasad,M,0,2025-10-12,337,BK-R50B-62,Road-650 Black- 62,487.0,R,2012-07-01,2008-12-27,Road Bikes,BI_RB,Bikes,Road Bikes,Yes,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89828,SO75123,FE-6654,18759,2014-01-28,2014-02-04,2014-02-09,22.0,1,22.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,485,FE-6654,Fender Set - Mountain,8.0,M,2013-07-01,1970-01-01,Mountain Bikes,BI_MB,Bikes,Mountain Bikes,Yes,NaN,NaT,NaN,NaN
89829,SO75123,ST-1401,18759,2014-01-28,2014-02-04,2014-02-09,159.0,1,159.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,486,ST-1401,All-Purpose Bike Stand,59.0,M,2013-07-01,1970-01-01,Bike Stands,AC_BS,Accessories,Bike Stands,No,NaN,NaT,NaN,NaN
89830,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,223,CA-1098,AWC Logo Cap,6.0,S,2011-07-01,2007-12-28,Caps,CL_CA,Clothing,Caps,No,NaN,NaT,NaN,NaN
89831,SO75123,CA-1098,18759,2014-01-28,2014-02-04,2014-02-09,9.0,1,9.0,18759,AW00018759,Devin,Phillips,S,M,2026-01-04,224,CA-1098,AWC Logo Cap,5.0,S,2012-07-01,2008-12-27,Caps,CL_CA,Clothing,Caps,No,NaN,NaT,NaN,NaN


In [63]:
# Remove redundant columns
CRM_ERP = CRM_ERP.drop(columns=['sls_prd_key', 'sls_cust_id', 'CID', 'SUBCAT', 'cst_gndr'])

# Remove zero values
CRM_ERP = CRM_ERP.replace(0, np.nan)
CRM_ERP_clean = CRM_ERP.dropna()
CRM_ERP_clean

,sls_ord_num,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price,cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_create_date,prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt,subcat,ID,CAT,MAINTENANCE,BDATE,GEN,CNTRY
1,SO43698,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Yes,1970-06-17,Female,France
2,SO43699,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Yes,1952-06-01,Female,United States
6,SO43702,2010-12-30,2011-01-06,2011-01-11,3578.0,1,3578.0,27645,AW00027645,Colin,Anand,M,2026-01-25,311,BK-R93R-44,Road-150 Red- 44,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes,1972-08-14,Male,United States
10,SO43706,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578.0,27621,AW00027621,Edward,Brown,S,2026-01-25,312,BK-R93R-48,Road-150 Red- 48,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes,1975-01-23,Male,United States
11,SO43707,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578.0,27616,AW00027616,Emma,Brown,S,2026-01-21,312,BK-R93R-48,Road-150 Red- 48,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes,1971-06-10,Female,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89741,SO75098,2014-01-28,2014-02-04,2014-02-09,2.0,1,2.0,23381,AW00023381,Bianca,Hu,M,2026-01-05,480,PK-7098,Patch Kit/8 Patches,1.0,S,2013-07-01,1970-01-01,Cleaners,AC_CL,Accessories,Yes,1967-01-08,Female,United States
89799,SO75114,2014-01-28,2014-02-04,2014-02-09,21.0,1,21.0,24704,AW00024704,Allen,Schmidt,M,2026-01-05,538,TI-R092,LL Road Tire,8.0,R,2013-07-01,1970-01-01,Road Bikes,BI_RB,Bikes,Yes,1983-03-10,Male,Germany
89800,SO75115,2014-01-28,2014-02-04,2014-02-09,21.0,1,21.0,26832,AW00026832,Kate,Xie,M,2026-01-14,538,TI-R092,LL Road Tire,8.0,R,2013-07-01,1970-01-01,Road Bikes,BI_RB,Bikes,Yes,1977-01-30,Female,Germany
89801,SO75115,2014-01-28,2014-02-04,2014-02-09,4.0,1,4.0,26832,AW00026832,Kate,Xie,M,2026-01-14,529,TT-R982,Road Tire Tube,1.0,R,2013-07-01,1970-01-01,Road Bikes,BI_RB,Bikes,Yes,1977-01-30,Female,Germany


In [64]:
# Fix column name format
CRM_ERP_clean = CRM_ERP_clean.rename(columns={
    'sls_ord_num': 'OrderNumber',
    'sls_order_dt': 'OrderDate',
    'sls_ship_dt': 'ShipDate',
    'sls_due_dt': 'DueDate',
    'sls_sales': 'SalesAmount',
    'sls_quantity': 'Quantity',
    'sls_price': 'UnitPrice',
    'cst_id': 'CustomerID',
    'cst_key': 'CustomerKey',
    'cst_firstname': 'FirstName',
    'cst_lastname': 'LastName',
    'cst_marital_status': 'MaritalStatus',
    'cst_create_date': 'CustomerCreateDate',
    'prd_id': 'ProductID',
    'prd_key': 'ProductKey',
    'prd_nm': 'ProductName',
    'prd_cost': 'ProductCost',
    'prd_line': 'ProductLine',
    'prd_start_dt': 'ProductStartDate',
    'prd_end_dt': 'ProductEndDate',
    'subcat': 'SubCategory',
    'ID': 'SubCategoryID',
    'CAT': 'Category',
    'MAINTENANCE': 'MaintenancePlan',
    'BDATE': 'BirthDate',
    'GEN': 'Gender',
    'CNTRY': 'Country'
})

# Reorder columns
desired_order = [
    'OrderNumber', 'OrderDate', 'ShipDate', 'DueDate', 'SalesAmount', 'Quantity', 'UnitPrice',
    'CustomerID', 'CustomerKey', 'FirstName', 'LastName', 'MaritalStatus', 'BirthDate', 'Gender', 'Country', 'CustomerCreateDate',
    'ProductID', 'ProductKey', 'ProductName', 'ProductCost', 'ProductLine', 'ProductStartDate', 'ProductEndDate',
    'SubCategory', 'SubCategoryID', 'Category', 'MaintenancePlan'
]
CRM_ERP_clean = CRM_ERP_clean[desired_order]

# Reset index
CRM_ERP_final = CRM_ERP_clean.reset_index(drop=True)
CRM_ERP_final

,OrderNumber,OrderDate,ShipDate,DueDate,SalesAmount,Quantity,UnitPrice,CustomerID,CustomerKey,FirstName,LastName,MaritalStatus,BirthDate,Gender,Country,CustomerCreateDate,ProductID,ProductKey,ProductName,ProductCost,ProductLine,ProductStartDate,ProductEndDate,SubCategory,SubCategoryID,Category,MaintenancePlan
0,SO43698,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,1970-06-17,Female,France,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Yes
1,SO43699,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,1952-06-01,Female,United States,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Yes
2,SO43702,2010-12-30,2011-01-06,2011-01-11,3578.0,1,3578.0,27645,AW00027645,Colin,Anand,M,1972-08-14,Male,United States,2026-01-25,311,BK-R93R-44,Road-150 Red- 44,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes
3,SO43706,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578.0,27621,AW00027621,Edward,Brown,S,1975-01-23,Male,United States,2026-01-25,312,BK-R93R-48,Road-150 Red- 48,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes
4,SO43707,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578.0,27616,AW00027616,Emma,Brown,S,1971-06-10,Female,United States,2026-01-21,312,BK-R93R-48,Road-150 Red- 48,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27161,SO75098,2014-01-28,2014-02-04,2014-02-09,2.0,1,2.0,23381,AW00023381,Bianca,Hu,M,1967-01-08,Female,United States,2026-01-05,480,PK-7098,Patch Kit/8 Patches,1.0,S,2013-07-01,1970-01-01,Cleaners,AC_CL,Accessories,Yes
27162,SO75114,2014-01-28,2014-02-04,2014-02-09,21.0,1,21.0,24704,AW00024704,Allen,Schmidt,M,1983-03-10,Male,Germany,2026-01-05,538,TI-R092,LL Road Tire,8.0,R,2013-07-01,1970-01-01,Road Bikes,BI_RB,Bikes,Yes
27163,SO75115,2014-01-28,2014-02-04,2014-02-09,21.0,1,21.0,26832,AW00026832,Kate,Xie,M,1977-01-30,Female,Germany,2026-01-14,538,TI-R092,LL Road Tire,8.0,R,2013-07-01,1970-01-01,Road Bikes,BI_RB,Bikes,Yes
27164,SO75115,2014-01-28,2014-02-04,2014-02-09,4.0,1,4.0,26832,AW00026832,Kate,Xie,M,1977-01-30,Female,Germany,2026-01-14,529,TT-R982,Road Tire Tube,1.0,R,2013-07-01,1970-01-01,Road Bikes,BI_RB,Bikes,Yes


In [65]:
CRM_ERP_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27166 entries, 0 to 27165
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   OrderNumber         27166 non-null  object        
 1   OrderDate           27166 non-null  datetime64[ns]
 2   ShipDate            27166 non-null  datetime64[ns]
 3   DueDate             27166 non-null  datetime64[ns]
 4   SalesAmount         27166 non-null  float64       
 5   Quantity            27166 non-null  int64         
 6   UnitPrice           27166 non-null  float64       
 7   CustomerID          27166 non-null  int64         
 8   CustomerKey         27166 non-null  object        
 9   FirstName           27166 non-null  object        
 10  LastName            27166 non-null  object        
 11  MaritalStatus       27166 non-null  object        
 12  BirthDate           27166 non-null  datetime64[ns]
 13  Gender              27166 non-null  object    

## **Data Loading**

Save cleaned data to Render (PostgreSQL) database with DuckDB extension.

In [67]:
# Connect to Data Warehouse
DataWarehouse_URL = os.getenv("DATA_WAREHOUSE_URL")
engine_dw = sqlalchemy.create_engine(DataWarehouse_URL, client_encoding='utf8')
connection_dw = engine_dw.connect()

In [68]:
# Save DataFrame to database
CRM_ERP_final.to_sql("crm_erp", con=connection_dw, if_exists="replace", index=False)

166

In [69]:
# Verify the data was saved correctly
query = "SELECT * FROM crm_erp LIMIT 5;"
check_df = pd.read_sql(query, con=connection_dw)
check_df

,OrderNumber,OrderDate,ShipDate,DueDate,SalesAmount,Quantity,UnitPrice,CustomerID,CustomerKey,FirstName,LastName,MaritalStatus,BirthDate,Gender,Country,CustomerCreateDate,ProductID,ProductKey,ProductName,ProductCost,ProductLine,ProductStartDate,ProductEndDate,SubCategory,SubCategoryID,Category,MaintenancePlan
0,SO43698,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,28389,AW00028389,Rachael,Martinez,S,1970-06-17,Female,France,2026-01-25,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Yes
1,SO43699,2010-12-29,2011-01-05,2011-01-10,3400.0,1,3400.0,25863,AW00025863,Sydney,Wright,S,1952-06-01,Female,United States,2026-01-14,346,BK-M82S-44,Mountain-100 Silver- 44,1912.0,M,2011-07-01,2007-12-28,Mountain Bikes,BI_MB,Bikes,Yes
2,SO43702,2010-12-30,2011-01-06,2011-01-11,3578.0,1,3578.0,27645,AW00027645,Colin,Anand,M,1972-08-14,Male,United States,2026-01-25,311,BK-R93R-44,Road-150 Red- 44,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes
3,SO43706,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578.0,27621,AW00027621,Edward,Brown,S,1975-01-23,Male,United States,2026-01-25,312,BK-R93R-48,Road-150 Red- 48,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes
4,SO43707,2010-12-31,2011-01-07,2011-01-12,3578.0,1,3578.0,27616,AW00027616,Emma,Brown,S,1971-06-10,Female,United States,2026-01-21,312,BK-R93R-48,Road-150 Red- 48,2171.0,R,2011-07-01,2007-12-28,Road Bikes,BI_RB,Bikes,Yes


In [70]:
# Verify the extensions
query_ext = "SELECT * FROM pg_extension;"
pd.read_sql(query_ext, con=connection_dw)

,oid,extname,extowner,extnamespace,extrelocatable,extversion,extconfig,extcondition
0,13564,plpgsql,10,11,False,1.0,None,None
1,16399,pg_duckdb,16385,2200,False,0.2.0,None,None


In [71]:
# Close connections and dispose engines
connection_dw.close()
engine_dw.dispose()